In [1]:
from utils import controlled_compute, logger, tqdmbar, show_dask_progress, load_zarr, ZARRLOC, permute_in_chunks
from typing import Generator, IO, Any, Tuple, List, Union, Dict, Optional
from scipy.sparse import csr_matrix, coo_matrix
import nexusformat.nexus as nx
import pandas as pd
import polars as pl
import numpy as np
import h5py
import sys
import os
import math
import zarr

In [2]:
# __init__.py at ./
# sys.path.append('../../scarf')
import sys
sys.path.append('../')
import scarf

Scarf is not installed


In [3]:
ds_ctrl = scarf.DataStore(
    '../../notebooks/scarf_datasets/kang_15K_pbmc_rnaseq/data.zarr/',
    nthreads=4
)
ds_stim = scarf.DataStore(
    '../../notebooks/scarf_datasets/kang_14K_ifnb-pbmc_rnaseq/data.zarr',
    nthreads=4
)

In [4]:
test_path = '../../tmp/kang_merged_pbmc_rnaseq_test.zarr'
!rm -rf {test_path}

In [5]:
merger = scarf.DatasetMerge(
    datasets = [ds_ctrl, ds_stim],
    zarr_path = test_path,
    names = ['ctrl', 'stim'],
)
merger.dump()

Dumping RNA


Writing data to merged file:   0%|                                                                            …

Writing data to merged file:   0%|                                                                            …

Merging complete


In [6]:
ds_ctrl = scarf.DataStore(
    '../../tmp/10k_Human_PBMC_TotalSeqB_3p_gemx_Multiplex_count_raw_feature_bc_matrix.zarr',
    nthreads=4,
    default_assay='RNA'
)
ds_stim = scarf.DataStore(
    '../../tmp/10k_Mouse_Neurons_3p_nextgem_Multiplex_count_raw_feature_bc_matrix.zarr',
    nthreads=4
)

In [7]:
test_path = '../../tmp/zarr-merged.zarr'
!rm -rf {test_path}

In [8]:
dataset_merger = scarf.DatasetMerge(
    datasets = [ds_ctrl, ds_stim],
    zarr_path = test_path,
    names = ['ctrl', 'stim'],
    overwrite = True
)
dataset_merger.dump(nthreads=4)

INFO: Generated dummy ADT assay for datastore DataStore has 10276 (10276) cells with 1 assays: RNA
   Cell metadata:
            'I', 'ids', 'names', 'RNA_nCounts', 'RNA_nFeatures', 
            'RNA_percentMito', 'RNA_percentRibo'
   RNA assay has 17991 (33696) features and following metadata:
            'I', 'ids', 'names', 'dropOuts', 'nCells', 
          
Dumping RNA


Writing data to merged file:   0%|                                                                            …

Writing data to merged file:   0%|                                                                            …

Dumping ADT


Writing data to merged file:   0%|                                                                            …

Writing data to merged file:   0%|                                                                            …

Merging complete


# Test

In [9]:
import zarr
import numpy as np
import math
from tqdm import tqdm

In [10]:
original = zarr.open('../../tmp/kang_merged_pbmc_rnaseq_org.zarr/RNA/counts')
original.info

Type,zarr.core.Array
Data type,uint32
Shape,"(29065, 35635)"
Chunk shape,"(1000, 1000)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=BITSHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,4142925100 (3.9G)
No. bytes stored,81289791 (77.5M)
Storage ratio,51.0


In [11]:
test = zarr.open('../../notebooks/tmp/kang_merged_pbmc_rnaseq_new.zarr/RNA/counts')
test.info

Type,zarr.core.Array
Data type,uint32
Shape,"(29065, 35635)"
Chunk shape,"(1000, 1000)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=BITSHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,4142925100 (3.9G)
No. bytes stored,81285726 (77.5M)
Storage ratio,51.0


In [12]:
assert test.shape == original.shape
assert test.dtype == original.dtype
assert test.nchunks == original.nchunks

In [13]:
nChunks = math.ceil(test.shape[0]/test.chunks[0])
nChunks

30

In [14]:
nChunks = math.ceil(test.shape[0]/test.chunks[0])
sumOrg = 0
sumTest = 0
for chunk in tqdm(range(nChunks)):
    sumOrg += np.sum(original.get_block_selection(chunk))
    sumTest += np.sum(test.get_block_selection(chunk))
sumOrg, sumTest

100%|██████████| 30/30 [00:10<00:00,  2.98it/s]


(48600925.0, 48600925.0)

In [15]:
assert sumOrg == sumTest

In [16]:
nChunks = math.ceil(test.shape[1]/test.chunks[1])
sumOrg = 0
sumTest = 0
for chunk in tqdm(range(nChunks)):
    sumOrg += np.sum(original.blocks[:, chunk])
    sumTest += np.sum(test.blocks[:, chunk])
sumOrg, sumTest

100%|██████████| 36/36 [00:08<00:00,  4.39it/s]


(48600925.0, 48600925.0)

In [17]:
assert sumOrg == sumTest

# Test 2

In [18]:
import zarr
import numpy as np
import math
from tqdm import tqdm

In [19]:
original = zarr.open('../../tmp/zarr-merged-org.zarr/RNA/counts')
original.info

Type,zarr.core.Array
Data type,uint32
Shape,"(21196, 72302)"
Chunk shape,"(1000, 1000)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=BITSHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,6130052768 (5.7G)
No. bytes stored,175706945 (167.6M)
Storage ratio,34.9


In [20]:
test = zarr.open('../../tmp/zarr-merged.zarr/RNA/counts')
test.info

Type,zarr.core.Array
Data type,uint32
Shape,"(21196, 72302)"
Chunk shape,"(1000, 1000)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=BITSHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,6130052768 (5.7G)
No. bytes stored,175700063 (167.6M)
Storage ratio,34.9


In [21]:
assert test.shape == original.shape
assert test.dtype == original.dtype
assert test.nchunks == original.nchunks

In [22]:
nChunks = math.ceil(test.shape[0]/test.chunks[0])
sumOrg = 0
sumTest = 0
for chunk in tqdm(range(nChunks)):
    sumOrg += np.sum(original.get_block_selection(chunk))
    sumTest += np.sum(test.get_block_selection(chunk))
sumOrg, sumTest

100%|██████████| 22/22 [00:10<00:00,  2.13it/s]


(267089481.0, 267089481.0)

In [23]:
assert sumOrg == sumTest

In [24]:
nChunks = math.ceil(test.shape[1]/test.chunks[1])
sumOrg = 0
sumTest = 0
for chunk in tqdm(range(nChunks)):
    sumOrg += np.sum(original.blocks[:, chunk])
    sumTest += np.sum(test.blocks[:, chunk])
sumOrg, sumTest

100%|██████████| 73/73 [00:09<00:00,  7.97it/s]


(267089481.0, 267089481.0)

In [25]:
assert sumOrg == sumTest